In [5]:
pip install biopython
pip install git+https://github.com/Benjamin-Lee/CodonAdaptationIndex.git

  Cloning https://github.com/Benjamin-Lee/CodonAdaptationIndex.git to /tmp/pip-req-build-fd_1t8bu
  Running command git clone --filter=blob:none --quiet https://github.com/Benjamin-Lee/CodonAdaptationIndex.git /tmp/pip-req-build-fd_1t8bu
  Resolved https://github.com/Benjamin-Lee/CodonAdaptationIndex.git to commit b6e017a92c58829f6a5aec8c26a21262bc2a6610
  Preparing metadata (setup.py) ... done
  Created wheel for CAI: filename=CAI-1.0.5.dev3+gb6e017a.d20240414-py3-none-any.whl size=7822 sha256=6dbd73a149ca17e84e7d8d41e7797ec802053de623384f5618f95b3e44e80875
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ubs5jzo/wheels/b7/62/7a/6b6b3df9a7e18e6d1cff29b703318c7607f833e330c59bfdbc
Successfully built CAI
Note: you may need to restart the kernel to use updated packages.


In [23]:
from CAI import CAI
from Bio.Seq import Seq
import numpy as np
import math

# Array of codons
codons = ["ATA", "ATG", "AAT", "AAC"]

num_positions = 3
num_codons = len(codons)

# Initialize empty NumPy array for storing CAI values
num_codons = len(codons)
cai_matrix = np.zeros((num_codons, num_codons))

# Iterate through each pair of codons
for i in range(num_codons):
    for j in range(i+1, num_codons):  # Only iterate over upper triangular part
        codon_i = codons[i]
        codon_j = codons[j]
        
        # Calculate CAI for codon_i with respect to codon_j
        cai_value = CAI(Seq(codon_i), reference=[Seq(codon_j)])
        if math.isnan(cai_value):
            cai_value = 1000
        # Store CAI value in the upper triangular part of the matrix
        cai_matrix[i, j] = cai_value

# Print the resulting CAI matrix
print("CAI Matrix:")
print(cai_matrix)

CAI Matrix:
[[0.e+00 1.e+00 1.e+00 1.e+00]
 [0.e+00 0.e+00 1.e+03 1.e+03]
 [0.e+00 0.e+00 0.e+00 5.e-01]
 [0.e+00 0.e+00 0.e+00 0.e+00]]


/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [29]:
def print_results(result):
    binary_vars = result.variables
    binary_values = result.x

    # Initialize an empty matrix
    matrix = np.zeros((num_codons, num_positions))

    # Populate the matrix with the binary variable values
    for value, var in zip(binary_values, binary_vars):
        if value == 1.0:
            parts = var.name.split('_')
            codon_index = int(parts[1])
            position_index = int(parts[2])
            matrix[codon_index, position_index] = 1

    # Print or visualize the matrix
    print(matrix)

In [30]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler

weights_matrix = np.array([[0.8, 0.3, 0.0, 0.0],  # Codon 1
                            [0.2, 0.5, 0.0, 0.0],  # Codon 2
                            [0.0, 0.5, 0.5, 0.5],
                           [0.0, 0.5, 0.5, 0.5]])  # Codon 3

binary_vars = []

qp = QuadraticProgram()
linear_coefficients = {}
quadratic_coefficients = {}
for i in range(num_codons):
    for j in range(num_positions):
        var = qp.binary_var(name=f'x_{i}_{j}')
        binary_vars.append(var)
        linear_coefficients[f'x_{i}_{j}'] = weights_matrix[i, j]
        if j > i:
            quadratic_coefficients[(f'x_{i}_{j-1}', f'x_{i+1}_{j}')] = cai_matrix[i, j]

print(linear_coefficients)
print(quadratic_coefficients)
        
for j in range(num_positions):
    constraint_dict = {var_name: 1.0 for var_name in [f'x_{i}_{j}' for i in range(num_codons)]}
    qp.linear_constraint(linear=constraint_dict, sense='==', rhs=1.0)

# Set the objective function to maximize
qp.maximize(linear=linear_coefficients, quadratic=quadratic_coefficients)

qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())
min_eigen_optimizer = MinimumEigenOptimizer(qaoa)
quantum_result = min_eigen_optimizer.solve(qp)
print("quantum results:")
print_results(quantum_result)


# Print the optimal solution
exact_mes = NumPyMinimumEigensolver()
exact = MinimumEigenOptimizer(exact_mes)
exact_result = exact.solve(qp)
print("classical result")
print_results(exact_result)

{'x_0_0': 0.8, 'x_0_1': 0.3, 'x_0_2': 0.0, 'x_1_0': 0.2, 'x_1_1': 0.5, 'x_1_2': 0.0, 'x_2_0': 0.0, 'x_2_1': 0.5, 'x_2_2': 0.5, 'x_3_0': 0.0, 'x_3_1': 0.5, 'x_3_2': 0.5}
{('x_0_0', 'x_1_1'): 1.0, ('x_0_1', 'x_1_2'): 1.0, ('x_1_1', 'x_2_2'): 1000.0}
quantum results:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
classical result
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
